In [24]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.manifold import TSNE
#from visualization import visualize, display, sequence_identity
from rdkit.Chem import AllChem
from scipy.stats import spearmanr, ttest_ind

def get_accuracy_level(predicted_ECs, true_ECs):
    """
    based on a list of predicted_ECs, calculates the highest level of accuracy achieved
    """
    #convert true_EC to a list
    if type(predicted_ECs) == str:
        predicted_ECs = [predicted_ECs]
        
    if type(true_ECs) == str:
        true_ECs = [true_ECs]

    maxes = []
    for true_EC in true_ECs:

        true_split = true_EC.split('.')
        
        counters = []
        for predicted_EC in predicted_ECs:
            #print(true_EC)
            predicted_split = predicted_EC.split('.')
            counter = 0
            
            for predicted, true in zip(predicted_split, true_split):
                if predicted == true:
                    counter += 1
                else:
                    break
            counters.append(counter)
            #print(counters)
        
        maxes.append(np.max(counters))
    return maxes
#import umap

#supress warnings
import warnings
warnings.filterwarnings("ignore")

### Task 1: If you alredy have an eval df that is in the correct format, code will look more like this

In [27]:
results = pd.DataFrame(columns=['baseline', 'split', 'k', 'level 4 accuracy', 'level 3 accuracy', 'level 2 accuracy', 'level 1 accuracy'])

#load the query_df that's already been generated
for baseline in ['Random', 'CLEAN']:
    for split in ['30', '30-50', 'price', 'promiscuous']:
        query_df = pd.read_csv('/disk1/jyang4/repos/CLEAN/app/results/protein_train/{}_protein_test_results_df.csv'.format(split))
        num_cols = 3
        #query_df = query_df.iloc[141:142,:]

        if baseline == 'Random':
            for i in range(query_df.shape[0]):
                np.random.seed(42)
                #shuffle the columns
                query_df.iloc[i, num_cols:] = query_df.iloc[i, num_cols:].sample(frac=1).values

        #collapse columns 0:3 into a single column list
        for k in [1, 5, 10]:
            query_df['predicted ECs'] = query_df.iloc[:, num_cols:num_cols+k].values.tolist()
            query_df['EC number list'] = query_df['EC number'].apply(lambda x: x.split(';'))
            
            query_df['k={} accuracy level'.format(k)] = query_df.apply(lambda x: get_accuracy_level(x['predicted ECs'],  x['EC number list']), axis=1)

        #make a k=1 retrieval accuracy table, broken down by accuracy greater than or equal to level 1, 2, 3, 4

        for k in [1, 5, 10]:
            accuracies = []
            for i in [4, 3, 2, 1]:
                exploded = query_df.explode('k={} accuracy level'.format(k))
                accuracy = exploded[exploded['k={} accuracy level'.format(k)] >= i].shape[0] / exploded.shape[0]
                accuracies.append(round(accuracy*100, 1))
                
            results.loc[len(results)] = [baseline, split, k, accuracies[0], accuracies[1], accuracies[2], accuracies[3]]

In [32]:
results[results['k'] == 5].sort_values(by='split')

,baseline,split,k,level 4 accuracy,level 3 accuracy,level 2 accuracy,level 1 accuracy
1,Random,30,5,0.0,0.6,12.6,58.9
13,CLEAN,30,5,70.9,80.0,85.1,92.6
4,Random,30-50,5,0.0,2.0,13.8,67.9
16,CLEAN,30-50,5,96.9,98.0,99.0,100.0
7,Random,price,5,0.0,4.7,12.1,59.7
19,CLEAN,price,5,61.1,83.2,95.3,98.0
10,Random,promiscuous,5,0.0,5.7,13.9,58.0
22,CLEAN,promiscuous,5,96.3,98.0,98.3,99.5
